In [2]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import torchvision
import torch
from torchvision import transforms
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import glob
import scipy.io
import pandas as pd
import shutil
import os
from numpy import *

In [5]:
filenames = glob.glob('ClothingAttributeDataset/labels/*.mat')

In [6]:
imagenames = glob.glob('ClothingAttributeDataset/images/*.jpg')

In [14]:
imagelist = []
for st in imagenames:
    imagelist.append(st[st.rindex('\\')+1 :-4])

In [15]:
df = pd.DataFrame()
df['imagename']  = imagelist

In [16]:
interval = [1.,2.,3.,4.,5.,6.,7.]

In [17]:
def extract_mat(filename):
    '''Extract the labels from the .mat file and load into a dataframe'''
    mat = scipy.io.loadmat(filename)
    mat = {k:v for k, v in mat.items() if k[0] != '_'}
    data = [float(j) for i in mat.values() for j in i]
    for i in range(len(data)):
        if data[i] not in interval:
            data[i] = np.nan
    data = data
    return data

In [18]:
for filename in filenames:
    
    st = filename
    df[st[st.rindex('\\')+1 :-4]] =  extract_mat(filename)
    

In [19]:
#df = df.fillna(0)
print(df)

     imagename  black_GT  blue_GT  brown_GT  category_GT  collar_GT  cyan_GT  \
0       000001       2.0      1.0       1.0          5.0        2.0      1.0   
1       000002       1.0      1.0       1.0          NaN        NaN      1.0   
2       000003       2.0      1.0       1.0          4.0        2.0      1.0   
3       000004       2.0      1.0       1.0          5.0        2.0      1.0   
4       000005       1.0      1.0       1.0          6.0        1.0      1.0   
...        ...       ...      ...       ...          ...        ...      ...   
1851    001852       1.0      1.0       1.0          1.0        2.0      1.0   
1852    001853       1.0      1.0       1.0          7.0        1.0      1.0   
1853    001854       1.0      1.0       1.0          7.0        2.0      1.0   
1854    001855       1.0      1.0       1.0          1.0        2.0      1.0   
1855    001856       1.0      2.0       1.0          7.0        1.0      2.0   

      gender_GT  gray_GT  green_GT  ...

In [21]:
values = {'gender_GT', 'skin_exposure_GT', 'sleevelength_GT', 'neckline_GT','category_GT'}

for v in values:
    df[v] = df[v].replace(np.nan, 0)


In [22]:
#Replaced all the NaN categories with 1

df = df.replace(np.nan, 1)
df.to_csv('df',index=False)

In [23]:
df = df.apply(pd.to_numeric)

In [25]:
df = df.drop('imagename',axis=1)

In [36]:
#подготовка датасета - создаю папки
import os
path = '/fin_df/'

for col in new_df.columns:
    os.mkdir(path+col)

In [49]:
#SORT PIC INTO FOLDERS
from PIL import Image

path = '/insta_df/images/'
for col in new_df.columns:
    col_list = new_df[col].to_list()
    for i in range(len(col_list)):
        if col_list[i] == 1:
            
            img = Image.open(imagenames[i]).resize((128,128)) 
            img.save( path +col+'/'+ imagelist[i]+'.jpg')
            
            
#sort for insta_df             
imagenames_inst = glob.glob('/insta_df/images/*.jpeg')
imagelist_inst = []
for st in imagenames_inst:
    imagelist_inst.append(st[st.rindex('\\')+1 :-5])

    
for i in imagenames_inst :   
    img = Image.open(i).resize((128,128)) 
    img.save(i)           

In [23]:
train_size = int(0.8 * len(full_dataset))
test_val_size = len(full_dataset) - train_size
train_dataset, test_val_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_val_size])
test_size = int(0.5 * len(test_val_dataset))
val_size = len(test_val_dataset) - test_size
test_dataset, val_dataset = torch.utils.data.random_split(test_val_dataset, [test_size, val_size])

In [25]:
batch_size = 32
train_batch_gen = torch.utils.data.DataLoader(train_dataset, 
                                              batch_size=batch_size,
                                              shuffle=True,
                                              num_workers=2)


val_batch_gen = torch.utils.data.DataLoader(val_dataset, 
                                              batch_size=batch_size,
                                              shuffle=True,
                                              num_workers=2)
test_batch_gen = torch.utils.data.DataLoader(test_dataset, 
                                              batch_size=batch_size,
                                              shuffle=True,
                                              num_workers=2)